# Optimizing Model Parameter
训练模型是一个迭代的过程，在每一次迭代中模型都会为输入数据生成对应的预测值，并且计算实际与预期之间的损失。随后会计算损失的导数，并且使用梯度下降的方法进行优化。

## Prerequisite Code

In [1]:
import torch
from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

training_data = datasets.FashionMNIST(
    root = '/work/Quickstart/data',
    train = True,
    download = False,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = '/work/Quickstart/data',
    train = False,
    download = True,
    transform = ToTensor()
)

batch_size = 64
train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
        

## Hyperparameters
超参数是用于控制模型优化过程的可调整的参数。

不同的超参数会影响模型的训练以及收敛速度

有几个超参数是经常关注的：
- Number of Epoch：训练的迭代次数
- Batch Size：在参数更新之前传递给网络的数据样本量
- Learning rate：在每个批次或迭代时，需要在一个什么样的程度上去更新模型的参数。过小的值会导致学习缓慢，过大的值可能导致模型丧失学习能力。

In [2]:
learning_rate = 1e-3
batch_size = 64
epoch = 5

## Optimization Loop
每一次 epoch 都会训练一次模型，并进行优化。包含 2 个部分
- 训练循环：迭代训练数据集，并尽可能的使优化参数收敛
- 验证/测试循环：迭代测试数据集，观察模型的效果是否提升。

### Loss Function
不同的损失函数，执行的任务不同

- `nn.MSELoss`即 Mean Square Error, 均方误差，主要用于回归任务
- `nn.NLLLoss`即 Negative Log Likelihood，负对数似然，主要用于分类任务
- `nn.CrossEntropyLoss`即交叉熵损失，其结合了 `nn.LogSoftMax` 和 `nn.NLLLoss`

In [3]:
# 初始化损失函数
loss_fn = nn.CrossEntropyLoss()

## Optimizer


In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

在训练循环内部，优化过程会发生在 3 个阶段
- 调用`optimizer.zero_grad()`用来重置模型参数的梯度。由于梯度会自动叠加，因此在每一次迭代中都需要将梯度清零
- 调用`loss.backward()`进行反向传播，存储参数的梯度
- 调用`optimizer.step()`通过梯度来调整参数

## Full implement

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) # dataloader有几个attribute？
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        # 反向传播
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f'loss: {loss:>7f} [{current:>5d} / {size:>5d}]')

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:8f} \n')

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

epochs = 10
for t in range(epochs):
    print(f'Epoch {t+1}\n-----------------------')
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
    print('Done!')

Epoch 1
-----------------------
loss: 2.297810 [   64 / 60000]
loss: 2.290495 [ 6464 / 60000]
loss: 2.274753 [12864 / 60000]
loss: 2.271425 [19264 / 60000]
loss: 2.243567 [25664 / 60000]
loss: 2.213969 [32064 / 60000]
loss: 2.223341 [38464 / 60000]
loss: 2.180729 [44864 / 60000]
loss: 2.180434 [51264 / 60000]
loss: 2.151636 [57664 / 60000]
Test Error: 
 Accuracy: 44.8%, Avg loss: 2.141100 

Done!
Epoch 2
-----------------------
loss: 2.141959 [   64 / 60000]
loss: 2.138960 [ 6464 / 60000]
loss: 2.076679 [12864 / 60000]
loss: 2.103532 [19264 / 60000]
loss: 2.044554 [25664 / 60000]
loss: 1.981585 [32064 / 60000]
loss: 2.019914 [38464 / 60000]
loss: 1.921700 [44864 / 60000]
loss: 1.926816 [51264 / 60000]
loss: 1.873215 [57664 / 60000]
Test Error: 
 Accuracy: 57.4%, Avg loss: 1.853471 

Done!
Epoch 3
-----------------------
loss: 1.876900 [   64 / 60000]
loss: 1.853269 [ 6464 / 60000]
loss: 1.723816 [12864 / 60000]
loss: 1.782639 [19264 / 60000]
loss: 1.674705 [25664 / 60000]
loss: 1.62238